In [42]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Latif-Calderón\AppData\Local\Temp\ipykernel_6360\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [44]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [45]:
display(data)

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
995,So what's the latest? It sounds contradictory ...,0
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1
997,Barb I will call to explain. Are you back in t...,0
998,Yang on travelNot free tonite.May work tomorrow,0


In [46]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_test.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 1)


In [47]:
display(data)

,text
0,usiness is for the fact that the deceased man ...
1,They are happy to adjust to the afternoon. I a...
2,Lael Brainard was confirmed 78-19 this afterno...
3,H <hrod17@clintonemail.com>Friday March 26 201...
4,"n;""> Dear Good Friend,<br><br><br>I am happy t..."
...,...
995,"Dear Friend,Do accept my sincere apologies if ..."
996,FROM THE DESK OF:MR.WANG QINHANG SENG BANK LTD...
997,He can speak at 7pm. He said that it does not ...
998,Mary Landrieu asked me to attend a dinner abou...


### Let's divide the training and test set into two partitions

In [82]:
# Your code
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the data
data = pd.read_csv("../data/kg_train.csv", encoding='latin-1')

# Reduce the dataset size for faster development
data = data.head(1000)
print(f"Shape of the data: {data.shape}")

# Fill in missing values with an empty string
data.fillna("", inplace=True)

# Assume your dataset has a 'label' column that you want to predict,
# and the rest of the columns are features
target_column = 'label'  # Replace with your actual target column name
features = data.drop(columns=[target_column])
target = data[target_column]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data['text'], 
    data ['label'], 
    #features, 
    #target, 
    test_size=0.2, 
    random_state=42  # 20% for testing
)

# Print the shape of each partition to confirm the split
print(f"Training features shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test features shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")

Shape of the data: (1000, 2)
Training features shape: (800,)
Training labels shape: (800,)
Test features shape: (200,)
Test labels shape: (200,)


## Data Preprocessing

In [83]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [84]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Initialize the stemmer
snowball = SnowballStemmer('english')

# Define the HTML cleaning function
def clean_html(content):
    # Remove inline JavaScript/CSS using regex
    content = re.sub(r'<(script|style).*?>.*?(</\1>)', '', content, flags=re.S)

    # Remove HTML comments
    content = re.sub(r'<!--.*?-->', '', content, flags=re.S)

    # Remove remaining HTML tags
    content = re.sub(r'<.*?>', '', content)

    # Strip extra spaces, newlines, and tabs
    content = re.sub(r'\s+', ' ', content).strip()

    # Remove punctuation
    content = content.translate(str.maketrans('', '', string.punctuation))

    # Convert to lowercase
    content = content.lower()

    # Tokenize and remove stopwords
    tokens = content.split()
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmed_words = [snowball.stem(word) for word in filtered_words]

    # Join the words back into a cleaned string
    cleaned_content = ' '.join(stemmed_words)

    return cleaned_content

# Load the data from the CSV file
data_path = '../data/kg_train.csv' # adjust the path to where your CSV file is
data = pd.read_csv(data_path, encoding='latin-1')  # Adjust encoding if necessary

# Assuming the text data is in a column named 'text', replace 'text' with the actual column name
data['cleaned_text'] = data['text'].apply(clean_html)

# Display the first few rows of the cleaned data
print(data[['text', 'cleaned_text']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strict privat busi propos mike chukwu...  
1                                                     
2  noracheryl email dozen memo haiti weekend plea...  
3  dear sir2fmadam2c know propos might surpris em...  
4                                                fyi  


In [85]:
# Your code
import re
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Initialize the stemmer
snowball = SnowballStemmer('english')

# Example HTML content
html_content = """
<!DOCTYPE html>
<html>
<head>
  <style>
    p {color: red;}
    body {background-color: lightblue;}
  </style>
  <script>
    console.log("Hello, World!");
  </script>
</head>
<body>
  <h1>This is a heading</h1>
  <p>This is a paragraph.</p>
  <!-- This is a comment -->
</body>
</html>
"""

def clean_html(content):
    # Remove inline JavaScript/CSS using regex
    content = re.sub(r'<(script|style).*?>.*?(</\1>)', '', content, flags=re.S)

    # Remove HTML comments
    content = re.sub(r'<!--.*?-->', '', content, flags=re.S)

    # Remove remaining HTML tags
    content = re.sub(r'<.*?>', '', content)

     # Strip extra spaces, newlines, and tabs
    html_content = re.sub(r'\s+', ' ', html_content).strip()

    # Optionally, you may want to remove punctuation or transform the text 
    content = content.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation

    # Convert to lowercase
    content = content.lower()

    # Tokenize and remove stopwords
    tokens = content.split()
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmed_words = [snowball.stem(word) for word in filtered_words]

    # Join the words back into a cleaned string
    cleaned_content = ' '.join(stemmed_words)

    return cleaned_content

cleaned_text = clean_html(html_content)
print(cleaned_text)

UnboundLocalError: local variable 'html_content' referenced before assignment

In [86]:
import re

def clean_html(html_content):
    # Remove inline JavaScript/CSS:
    # Matches anything inside `<script>` or `<style>`, including new lines
    html_content = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)

    # Remove HTML comments:
    # Matches anything inside `<!-- ... -->`
    html_content = re.sub(r'<!--.*?-->', '', html_content, flags=re.DOTALL)

    # Remove remaining HTML tags:
    # Matches `<tag>` while allowing handling of mismatched HTML like `<img src='...'/>`
    html_content = re.sub(r'<.*?>', '', html_content)

    # Strip extra spaces, newlines, and tabs
    html_content = re.sub(r'\s+', ' ', html_content).strip()

    return html_content

# Example HTML content
html_data = """
<!DOCTYPE html>
<html>
<head>
  <style>
    p {color: red;}
    body {background-color: lightblue;}
  </style>
  <script type="text/javascript">
    console.log("Hello, World!");
  </script>
</head>
<body>
  <h1>This is a heading</h1>
  <p>This is a paragraph.</p>
  <!-- This is a comment -->
</body>
</html>
"""

# Clean the HTML
cleaned_text = clean_html(html_data)
print(cleaned_text)

This is a heading This is a paragraph.


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [87]:
import pandas as pd
import re

def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Remove numbers specifically, if any remain
    text = re.sub(r'\d+', '', text)

    # Remove single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Optionally remove prefixed 'b' if you are dealing with byte strings
    text = text.replace("b'", '')  # Adjust according to actual data format
    
    # Convert to lowercase
    text = text.lower()

    return text

# Adjust the path to where your CSV file is located
data_path = '../data/kg_train.csv'

# Load the data
df = pd.read_csv(data_path, encoding='latin-1')

# Assuming the text data is in a column named 'text' (adjust this if necessary)
df['cleaned_text'] = df['text'].apply(clean_text)

# Display the first few rows of the cleaned data
print(df[['text', 'cleaned_text']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strictly private business proposal am...  
1                                            will do  
2  noracheryl has emailed dozens of memos about h...  
3  dear sirfmadamc know that this proposal might ...  
4                                                fyi  


In [54]:
# Your code
import re

def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Remove numbers specifically, if any remain
    text = re.sub(r'\d+', '', text)

    # Remove single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Optionally remove prefixed 'b' if you are dealing with byte strings
    text = re.sub(r"\bb'", '', text)  # This depends on the actual input format
    
    # Convert to lowercase
    text = text.lower()

    return text

# Sample text for demonstration
sample_text = """
This is a sample text with numbers 123 and special characters! Let's clean this up:
Inline styles, <b>bold</b>, single characters like a, numbers such as 7, etc.
Prefix such as b'Sample prefixed binary string.'
"""

# Clean the text
cleaned_text = clean_text(sample_text)
print(cleaned_text)

this is sample text with numbers and special characters lets clean this up inline styles bboldb single characters like numbers such as etc prefix such as bsample prefixed binary string


## Now let's work on removing stopwords
Remove the stopwords.

In [89]:
import pandas as pd
import re

def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Remove numbers specifically, if any remain
    text = re.sub(r'\d+', '', text)

    # Remove single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Optionally remove prefixed 'b' if you are dealing with byte strings
    text = text.replace("b'", '')  # Adjust according to actual data format
    
    # Convert to lowercase
    text = text.lower()

    return text

# Adjust the path to where your CSV file is located
data_path = '../data/kg_train.csv'

# Load the data
df = pd.read_csv(data_path, encoding='latin-1')

# Assuming the text data is in a column named 'text' (adjust this if necessary)
df['cleaned_text'] = df['text'].apply(clean_text)

# Display the first few rows of the cleaned data
print(df[['text', 'cleaned_text']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strictly private business proposal am...  
1                                            will do  
2  noracheryl has emailed dozens of memos about h...  
3  dear sirfmadamc know that this proposal might ...  
4                                                fyi  


In [56]:
import re
from nltk.corpus import stopwords

# Ensure NLTK stopwords are downloaded
import nltk
nltk.download('stopwords')

def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Remove numbers specifically, if any remain
    text = re.sub(r'\d+', '', text)

    # Remove single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Convert to lowercase
    text = text.lower()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_text = ' '.join([word for word in tokens if word not in stop_words])

    return filtered_text

# Sample text for demonstration
sample_text = """
This is a sample text with numbers 123 and special characters! Let's clean this up:
Inline styles, <b>bold</b>, single characters like a, numbers such as 7, etc.
Prefix such as b'Sample prefixed binary string.'
"""

# Clean the text
cleaned_text = clean_text(sample_text)
print("\nclean text:", cleaned_text)


clean text: sample text numbers special characters lets clean inline styles bboldb single characters like numbers etc prefix bsample prefixed binary string


[nltk_data] Downloading package stopwords to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [90]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary resources are downloaded from NLTK
nltk.download('punkt')  # For tokenization
nltk.download('wordnet')  # WordNet Lemmatizer needs this
nltk.download('stopwords')  # For stopwords

def clean_and_lemmatize(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Convert text to lowercase
    text = text.lower()

    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)

    # Initialize the WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize words (removing stopwords if desired)
    stop_words = set(stopwords.words('english'))
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(lemmatized_words)

# Adjust the path to where your CSV file is located
data_path = '../data/kg_train.csv'  # Change this to the correct file path

# Load the data
df = pd.read_csv(data_path, encoding='latin-1')  # Ensure the correct encoding

# Assuming the text data is in a column named 'text', adjust if necessary
df['cleaned_text'] = df['text'].apply(clean_and_lemmatize)

# Display the first few rows of the cleaned data
print(df[['text', 'cleaned_text']].head())

[nltk_data] Downloading package punkt to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  noracheryl emailed dozen memo haiti weekend pl...  
3  dear sirfmadamc know proposal might surprise e...  
4                                                fyi  


In [64]:
# Your code
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary resources are downloaded from NLTK
nltk.download('punkt')  # For tokenization
nltk.download('wordnet')  # WordNet Lemmatizer needs this
nltk.download('stopwords')  # For stopwords

def clean_and_lemmatize(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Convert text to lowercase
    text = text.lower()

    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)

    # Initialize the WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize words (removing stopwords if desired)
    stop_words = set(stopwords.words('english'))
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(lemmatized_words)

# Sample text for demonstration
sample_text = """
Running through the forest, the adventurous foxes and their fellow animals were enjoying a playful and carefree afternoon.
"""

# Clean and lemmatize the text
cleaned_lemmatized_text = clean_and_lemmatize(sample_text)
print(cleaned_lemmatized_text)

running forest adventurous fox fellow animal enjoying playful carefree afternoon


[nltk_data] Downloading package punkt to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [94]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

def preprocess_text(text):
    # Removing special characters and converting to lowercase
    text = re.sub(r'[^A-Za-z\s]', '', text).lower()
    return text

def get_top_words(messages, top_n=10):
    # Concatenate all messages
    all_words = ' '.join(messages)

    # Tokenize words
    words = all_words.split()

    # Remove stopwords and get the frequency distribution
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # Use Counter to get the most common words
    word_counts = Counter(filtered_words)
    return word_counts.most_common(top_n)

# Load your data from the CSV file
data_path = '../data/kg_train.csv'  # Change this to the path where your CSV is located

# Load the data
df = pd.read_csv(data_path, encoding='latin-1')  # Ensure correct encoding if necessary

# Debugging step: print column names to identify correct ones
print("Column names:", df.columns)

# Assuming there are columns named 'text' and 'label', adjust based on actual CSV
# Update these lines based on column names
text_column = 'text'  # Replace 'text' with actual column name with text data
label_column = 'label'  # Replace 'label' with actual column name with label data

df['cleaned_text'] = df[text_column].apply(preprocess_text)

# Separate ham and spam messages
ham_messages = df[df[label_column] == 'ham']['cleaned_text']
spam_messages = df[df[label_column] == 'spam']['cleaned_text']

# Get top words for ham and spam
top_ham_words = get_top_words(ham_messages, top_n=10)
top_spam_words = get_top_words(spam_messages, top_n=10)

print("Top 10 words in ham messages:", top_ham_words)
print("Top 10 words in spam messages:", top_spam_words)

[nltk_data] Downloading package stopwords to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Column names: Index(['text', 'label'], dtype='object')
Top 10 words in ham messages: []
Top 10 words in spam messages: []


In [65]:
# Your code
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

def preprocess_text(text):
    # Removing special characters and converting to lowercase
    text = re.sub(r'[^A-Za-z\s]', '', text).lower()
    return text

def get_top_words(messages, top_n=10):
    # Concatenate all messages
    all_words = ' '.join(messages)

    # Tokenize words
    words = all_words.split()

    # Remove stopwords and get the frequency distribution
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # Use Counter to get the most common words
    word_counts = Counter(filtered_words)
    return word_counts.most_common(top_n)

# Sample dataframe simulating loaded dataset
data = {
    'label': ['ham', 'spam', 'ham', 'spam', 'ham'],
    'message': [
        "Hello, how are you today?",
        "Win money now! Special offer just for you!!",
        "Let's meet up for lunch tomorrow.",
        "Congratulations, you've won a free gift card.",
        "Are we still on for golf this weekend?"
    ]
}

df = pd.DataFrame(data)

# Preprocess messages
df['cleaned_message'] = df['message'].apply(preprocess_text)

# Separate ham and spam messages
ham_messages = df[df['label'] == 'ham']['cleaned_message']
spam_messages = df[df['label'] == 'spam']['cleaned_message']

# Get top words for ham and spam
top_ham_words = get_top_words(ham_messages, top_n=10)
top_spam_words = get_top_words(spam_messages, top_n=10)

print("Top 10 words in ham messages:", top_ham_words)
print("Top 10 words in spam messages:", top_spam_words)

Top 10 words in ham messages: [('hello', 1), ('today', 1), ('lets', 1), ('meet', 1), ('lunch', 1), ('tomorrow', 1), ('still', 1), ('golf', 1), ('weekend', 1)]
Top 10 words in spam messages: [('win', 1), ('money', 1), ('special', 1), ('offer', 1), ('congratulations', 1), ('youve', 1), ('free', 1), ('gift', 1), ('card', 1)]


[nltk_data] Downloading package stopwords to C:\Users\Latif-
[nltk_data]     Calderón\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Extra features

In [95]:
import pandas as pd

# Sample dataframe (replace this with your actual data)
data_train = pd.DataFrame({
    'preprocessed_text': [
        "win a free iphone now",
        "hello, let's catch up over coffee tomorrow",
        "transfer your funds to this account for cheap",
        "enjoy the best euro holiday deals",
        "your bank account has been suspended"
    ]
})

data_val = pd.DataFrame({
    'preprocessed_text': [
        "cheap medication for sale",
        "meeting at the bank tomorrow",
        "urgent password reset required"
    ]
})

# Define patterns for monetary symbols and suspicious words
money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", 
                             "transfer", "transaction", "win", "deposit", "password"])

# Add new features for training data
data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list).astype(int)
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words).astype(int)
data_train['text_len'] = data_train['preprocessed_text'].apply(len)

# Add new features for validation data
data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list).astype(int)
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words).astype(int)
data_val['text_len'] = data_val['preprocessed_text'].apply(len)

# Display the augmented training data
print(data_train.head())

                               preprocessed_text  money_mark  \
0                          win a free iphone now           1   
1     hello, let's catch up over coffee tomorrow           1   
2  transfer your funds to this account for cheap           1   
3              enjoy the best euro holiday deals           1   
4           your bank account has been suspended           1   

   suspicious_words  text_len  
0                 1        21  
1                 0        42  
2                 1        45  
3                 0        33  
4                 1        36  


In [96]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,preprocessed_text,money_mark,suspicious_words,text_len
0,win a free iphone now,1,1,21
1,"hello, let's catch up over coffee tomorrow",1,0,42
2,transfer your funds to this account for cheap,1,1,45
3,enjoy the best euro holiday deals,1,0,33
4,your bank account has been suspended,1,1,36


## How would work the Bag of Words with Count Vectorizer concept?

In [97]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Load the data from the CSV file
data_path = '../data/kg_train.csv'  # Adjust path according to your folder structure
df = pd.read_csv(data_path, encoding='latin-1')  # Adjust encoding if necessary

# Print columns to check names
print("Column names in CSV:", df.columns)

# Assuming the text data is in a column named 'text'
# Replace 'text' with the actual column name containing text data
text_column = 'text'  # Update this with the correct column name
documents = df[text_column].dropna().tolist()  # Convert to a list of strings

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the documents
X = vectorizer.fit_transform(documents)

# Get feature names, which are the unique words in the corpus
feature_names = vectorizer.get_feature_names_out()

# Convert to an array for easy viewing
word_count_array = X.toarray()

# Display the matrix
print("Feature names:", feature_names)
print("Word count matrix:\n", word_count_array)

# Each entry in word_count_array corresponds to a count of each feature_name in a document.

Column names in CSV: Index(['text', 'label'], dtype='object')
Feature names: ['00' '000' '0000' ... 'â½x60ã' 'â½x70' 'â½ã']
Word count matrix:
 [[1 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [68]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Sample data
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Never jump over the lazy dog quickly.",
    "The quick brown fox wins the race by jumping high.",
]

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the documents
X = vectorizer.fit_transform(documents)

# Get feature names, which are the unique words in the corpus
feature_names = vectorizer.get_feature_names_out()

# Convert to an array for easy viewing
word_count_array = X.toarray()

# Display the matrix
print("Feature names:", feature_names)
print("Word count matrix:\n", word_count_array)

# Here's how you can interpret the feature_names and word_count_array:
# Each entry in word_count_array corresponds to a count of each feature_name in a document.

Feature names: ['brown' 'by' 'dog' 'fox' 'high' 'jump' 'jumping' 'jumps' 'lazy' 'never'
 'over' 'quick' 'quickly' 'race' 'the' 'wins']
Word count matrix:
 [[1 0 1 1 0 0 0 1 1 0 1 1 0 0 2 0]
 [0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0]
 [1 1 0 1 1 0 1 0 0 0 0 1 0 1 2 1]]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [100]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the data from the CSV file
data_path = '../data/kg_train.csv'  # Adjust path to the location of your file
df = pd.read_csv(data_path, encoding='latin-1')  # Adjust encoding if necessary

# Print columns to check their names
print("Column names in CSV:", df.columns)

# Assuming the text data is in a column named 'text'
# Replace 'text' with the actual column name containing text data
text_column = 'text'  # Update this with the correct column name
documents = df[text_column].dropna().tolist()  # Convert to a list, remove NaN entries

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the documents into TF-IDF vectors
X_tfidf = tfidf_vectorizer.fit_transform(documents)

# Print shape of the vectorized dataset
print("Shape of TF-IDF vectorized dataset:", X_tfidf.shape)

# Get and print feature names
feature_names = tfidf_vectorizer.get_feature_names_out()
print("TF-IDF Feature Names:", feature_names)

# Convert matrix to array for easier inspection (optional, can be large)
tfidf_array = X_tfidf.toarray()

# Print the dense matrix representation, might be largeprint("TF-IDF Feature Names:", feature_names)
print("TF-IDF Dense Array:\n", tfidf_array[:5])  # Print only first 5 rows as a sample

Column names in CSV: Index(['text', 'label'], dtype='object')
Shape of TF-IDF vectorized dataset: (5964, 79210)
TF-IDF Feature Names: ['00' '000' '0000' ... 'â½x60ã' 'â½x70' 'â½ã']
TF-IDF Dense Array:
 [[0.03375527 0.06271964 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.02894602 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [62]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample data
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Never jump over the lazy dog quickly.",
    "The quick brown fox wins the race by jumping high.",
]

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the documents into TF-IDF vectors
X_tfidf = tfidf_vectorizer.fit_transform(documents)

# Print shape of the vectorized dataset
print("Shape of TF-IDF vectorized dataset:", X_tfidf.shape)

# It's often useful to view the feature names (vocabulary)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Convert to array for illustration
tfidf_array = X_tfidf.toarray()

# Print the TF-IDF feature names and the dense representation
print("TF-IDF Feature Names:", feature_names)
print("TF-IDF Dense Array:\n", tfidf_array)

Shape of TF-IDF vectorized dataset: (3, 16)
TF-IDF Feature Names: ['brown' 'by' 'dog' 'fox' 'high' 'jump' 'jumping' 'jumps' 'lazy' 'never'
 'over' 'quick' 'quickly' 'race' 'the' 'wins']
TF-IDF Dense Array:
 [[0.31401745 0.         0.31401745 0.31401745 0.         0.
  0.         0.41289521 0.31401745 0.         0.31401745 0.31401745
  0.         0.         0.48772512 0.        ]
 [0.         0.         0.33729513 0.         0.         0.44350256
  0.         0.         0.33729513 0.44350256 0.33729513 0.
  0.44350256 0.         0.26193976 0.        ]
 [0.2667197  0.35070436 0.         0.2667197  0.35070436 0.
  0.35070436 0.         0.         0.         0.         0.2667197
  0.         0.35070436 0.41426329 0.35070436]]


## And the Train a Classifier?

In [75]:
# Classifier will make prediction on my data.
# Train the classifier (MultinomialNB)

model = MultinomialNB()

model.fit(X_train_tfidf, data_train['label'])

# Make predictions on the validation set

y_pred = model.predict(X_val_tfidf)



NameError: name 'MultinomialNB' is not defined

In [76]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import pandas as pd

# Sample dataset - replace this with your actual dataset
data = {
    'text': [
        "This is a sample text with some special words and phrases.",
        "Another text to showcase this classifier.",
        "We are going to predict if there's a pattern or not.",
        "Learning is fun with examples.",
        "Data can tell many things if observed well."
    ],
    'label': [0, 1, 0, 1, 0]  # Example labels
}

df = pd.DataFrame(data)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform the training and validation data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Initialize and train the Multinomial Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Make predictions on the validation set
predictions = model.predict(X_val_tfidf)

# Output predictions
print("Predictions:", predictions)

Predictions: [0]


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Assume we have the following sample dataset
data = {
    'text': [
        "Get a free trial now",
        "Join us for the meeting tomorrow",
        "Congratulations, you've won a prize",
        "Please confirm your email address",
        "Check out this amazing offer"
    ],
    'label': [1, 0, 1, 0, 1]  # Example labels: 1 for spam, 0 for ham
}

df = pd.DataFrame(data)

# Split the dataset into a training and validation set
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Initialize a TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform the training and validation data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Initialize the Multinomial Naive Bayes classifier
model = MultinomialNB()

# Train the classifier
model.fit(X_train_tfidf, y_train)

# Make predictions on the validation set
predictions = model.predict(X_val_tfidf)

# Evaluate the classifier
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_val, predictions))

Accuracy: 0.00
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



c:\Users\Latif-Calderón\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Latif-Calderón\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Latif-Calderón\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [101]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re

def preprocess_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)
    return text.lower()

# Load your data from the CSV file
data_path = '../data/kg_train.csv'  # Adjust the path as needed
df = pd.read_csv(data_path, encoding='latin-1')  # Adjust encoding if necessary

# Print column names to verify correct usage
print("Column names in CSV:", df.columns) 

# Assume the column with text data is labeled 'text' and labels are 'label'
# Update these variables with actual column names from your CSV
text_column = 'text'  # Update with actual text column name
label_column = 'label'  # Update with actual label column name

# Preprocess the text column
df[text_column] = df[text_column].apply(preprocess_text)

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(df[text_column], df[label_column], test_size=0.2, random_state=42)

# Explore different vectorizers
vectorizers = {
    'tfidf_unigram': TfidfVectorizer(),
    'tfidf_bigram': TfidfVectorizer(ngram_range=(1, 2)),
    'count_unigram': CountVectorizer(),
    'count_bigram': CountVectorizer(ngram_range=(1, 2)),
}

best_accuracy = 0
best_vectorizer = None
best_model = None

# Train and evaluate with each vectorizer
for name, vectorizer in vectorizers.items():
    # Transform datasets
    X_train_vec = vectorizer.fit_transform(X_train)
    X_val_vec = vectorizer.transform(X_val)
    
    # Initialize and train the classifier
    model = MultinomialNB()
    model.fit(X_train_vec, y_train)
    
    # Make predictions
    predictions = model.predict(X_val_vec)
    
    # Evaluate performance
    accuracy = accuracy_score(y_val, predictions)
    print(f"Accuracy with {name}: {accuracy:.2f}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_vectorizer = name
        best_model = model

print(f"Best feature representation: {best_vectorizer} with accuracy of {best_accuracy:.2f}")

Column names in CSV: Index(['text', 'label'], dtype='object')
Accuracy with tfidf_unigram: 0.93
Accuracy with tfidf_bigram: 0.90
Accuracy with count_unigram: 0.93
Accuracy with count_bigram: 0.94
Best feature representation: count_bigram with accuracy of 0.94


In [78]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import re

# Load your data from Kaggle or a local CSV file
# For demonstration purposes, I'm creating a mock dataset
data = {
    'text': [
        "Free trial offer just for you", 
        "Join the meeting at noon", 
        "Congratulations, you won!", 
        "Please confirm your email subscription", 
        "Earn $$ with this offer!"
    ],
    'label': [1, 0, 1, 0, 1]  # Example labels: 1 for spam, 0 for ham
}

df = pd.DataFrame(data)

# Basic text preprocessing function
def preprocess_text(text):
    # Remove special characters, numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)
    return text.lower()

# Apply preprocessing to the dataset
df['text'] = df['text'].apply(preprocess_text)

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Explore different vectorizers
vectorizers = {
    'tfidf_unigram': TfidfVectorizer(),
    'tfidf_bigram': TfidfVectorizer(ngram_range=(1, 2)),
    'count_unigram': CountVectorizer(),
    'count_bigram': CountVectorizer(ngram_range=(1, 2)),
}

best_accuracy = 0
best_vectorizer = None
best_model = None

# Train and evaluate with each vectorizer
for name, vectorizer in vectorizers.items():
    # Transform datasets
    X_train_vec = vectorizer.fit_transform(X_train)
    X_val_vec = vectorizer.transform(X_val)
    
    # Initialize and train the classifier
    model = MultinomialNB()
    model.fit(X_train_vec, y_train)
    
    # Make predictions
    predictions = model.predict(X_val_vec)
    
    # Evaluate performance
    accuracy = accuracy_score(y_val, predictions)
    print(f"Accuracy with {name}: {accuracy:.2f}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_vectorizer = name
        best_model = model

print(f"Best feature representation: {best_vectorizer} with accuracy of {best_accuracy:.2f}")

Accuracy with tfidf_unigram: 0.00
Accuracy with tfidf_bigram: 0.00
Accuracy with count_unigram: 0.00
Accuracy with count_bigram: 0.00
Best feature representation: None with accuracy of 0.00
